In [1]:
import numpy as np
import time
from scipy import signal
from scipy.signal import butter, sosfilt, sosfreqz
from sklearn.model_selection import KFold
import scipy.io as sio

In [2]:
crossvalidation = False
data_path 	= 'dataset/'
svm_kernel	= 'linear' #'sigmoid'#'linear' # 'sigmoid', 'rbf', 'poly'
svm_c 	= 0.1 # 0.05 for linear, 20 for rbf, poly: 0.1


useCSP = True


NO_splits = 5 # number of folds in cross validation
fs = 250. # sampling frequency
NO_channels = 22 # number of EEG channels
NO_subjects = 9


NO_csp = 24 # Total number of CSP feature per band and timewindow


bw = np.array([2,4,8,16,32]) # bandwidth of filtered signals
# self.bw = np.array([1,2,4,8,16,32])
ftype = 'butter' # 'fir', 'butter'
forder= 2 # 4

load_filterbank

In [3]:
max_freq = 40
order = 4

f_bands = np.zeros((99,2)).astype(float)

band_counter = 0
for bw in bw:
    startfreq = 4
    while (startfreq + bw <= max_freq):
        f_bands[band_counter] = [startfreq, startfreq + bw]

        if bw ==1: # do 1Hz steps
            startfreq = startfreq +1
        elif bw == 2: # do 2Hz steps
            startfreq = startfreq +2
        else : # do 4 Hz steps if Bandwidths >= 4Hz
            startfreq = startfreq +4

        band_counter += 1

# convert array to normalized frequency
f_bands_nom = 2*f_bands[:band_counter]/fs

In [4]:
n_bands = f_bands_nom.shape[0]

if ftype == 'butter':
    filter_bank = np.zeros((n_bands,order,6))
elif ftype == 'fir':
    filter_bank = np.zeros((n_bands,order))



for band_idx in range(n_bands):
    if ftype == 'butter':
        filter_bank[band_idx] = butter(order, f_bands_nom[band_idx], analog=False, btype='band', output='sos')
    elif ftype == 'fir':
        filter_bank[band_idx] = signal.firwin(order,f_bands_nom[band_idx],pass_zero=False)

In [5]:
time_windows_flt = np.array([
                        [2.5,3.5],
                        [3,4],
                        [3.5,4.5],
                        [4,5],
                        [4.5,5.5],
                        [5,6],
                        [2.5,4.5],
                        [3,5],
                        [3.5,5.5],
                        [4,6],
                        [2.5,6]])*fs # time windows in [s] x fs for using as a feature

time_windows = time_windows_flt.astype(int)

NO_bands = filter_bank.shape[0]
NO_time_windows = int(time_windows.size/2)
NO_features = NO_csp*NO_bands*NO_time_windows
train_time = 0
train_trials = 0
eval_time = 0
eval_trials = 0

In [6]:
print("Number of used features: "+ str(NO_features))
success_tot_sum = 0


if crossvalidation:
  print("Cross validation run")
else:
  print("Test data set")

start = time.time()

Number of used features: 11352
Test data set


CSP_Model ----> load_data

In [7]:
def get_data(subject,training,PATH):

	NO_channels = 22
	NO_tests = 6*48
	Window_Length = 7*250

	class_return = np.zeros(NO_tests)
	data_return = np.zeros((NO_tests,NO_channels,Window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(PATH+'A0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(PATH+'A0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2=[a_data1[0,0]]
		a_data3=a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_fs 		= a_data3[3]
		a_classes 	= a_data3[4]
		a_artifacts = a_data3[5]
		a_gender 	= a_data3[6]
		a_age 		= a_data3[7]
		for trial in range(0,a_trial.size):
			if(a_artifacts[trial]==0):
				data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:22])
				class_return[NO_valid_trial] = int(a_y[trial])
				NO_valid_trial +=1


	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]

In [8]:
def load_data(subject):
        if crossvalidation:
            data,label = get_data(subject ,True, data_path)
            kf = KFold(n_splits=NO_splits)
            split = 0
            for train_index, test_index in kf.split(data):
                if split == split:
                    train_data = data[train_index]
                    train_label = label[train_index]
                    eval_data = data[test_index]
                    eval_label = label[test_index]
                split += 1
        else:
            train_data, train_label = get_data(subject, True, data_path)
            eval_data, eval_label = get_data(subject, False, data_path)
            
        return train_data, train_label, eval_data, eval_label


In [9]:
for subject in range(1,2):
    train_data, train_label, eval_data, eval_label = load_data(subject)

In [10]:
train_data.shape

(273, 22, 1750)